This is kind of like a tutorial on how to make a controller. The nice thing is that it's a video and you can follow along with the slides and repo. Heres the [link](https://www.youtube.com/watch?v=J0n4n2RtbGs) to it.

So for my CRD I am going to have the PodCustomizer. This resource will either take pods without an owner reference and promote it to a deployment or delete depending on the option you have chosen. The CRD can look something like this:

In [ ]:
apiVersion: apiextensions.k8s.io/v1
kind: CustomResourceDefinition
metadata:
    name: podcustomizer.gr1nx.dev # resource and group
spec:
    group: gr1nx.dev
    versions:
        - name: v1
          served: true      # tells the apiServer that an endpoint is needed for this resource
          storage: true     # tells the apiServer to store the object in ETCD
          shceme:
            openAPIV3Schema:
                type: object
                properties: # fields inside the spec
                    spec:
                        type: object
                        properties:
                            promote:
                                type: bool
                    status: # this can contain an array of the names of the pods promoted or destroyed
                        type: object
                        properties:
                            numPromoted:
                                type: integer
                            numDestroyed:
                                type: integer
                            observedGeneration: # this counts how many times we've operated on a specific resource and if we need to operate again
                                type: integer
                                format: int64    
            subresources:
                status: {}
            additionalPrinterColumns:
            - name: promoter
              type: bool
              jsonPath: .spec.promote
            - name: podsDestroyed
              type: integer
              jsonPath: .status.numDestroyed
            - name: podsPromoted
              type: integer
              jsonPath: .status.numPromoted
     scope: Namespaced
     names:
        plural: podcustomizers
        singular: podcustomizer
        kind: PodCustomizer

Here's a [link](https://github.com/eddiezane/that-conference-k8s-controller) to the repo that is used in the talk. Most of it is copy paste from other controllers so its pretty dry. I'll write about what changes we are actually making though! The github repo contains all the code that is used in the talk and it also uses GCP. I wanna see if we can make our PodCustomizer controller and then port what this guy did to AWS.